In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import glob
from collections import deque, defaultdict
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [3]:
import warnings

# Suppress specific FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn._oldcore")


<IPython.core.display.Javascript object>

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

<IPython.core.display.Javascript object>

In [5]:
from sklearn.preprocessing import StandardScaler

<IPython.core.display.Javascript object>

In [6]:
def read_csv_files_grouped(csv_files, plant, header=[0, 1]):
    results = []
    for filepath in csv_files:
        df = pd.read_csv(filepath, header=header, low_memory=False)
        df["plant"] = plant  # Add the 'plant' column
        results.append(df)
    df = pd.concat(results, ignore_index=True)
    return df

<IPython.core.display.Javascript object>

In [7]:
def read_csv_files_full(csv_files, plant):
    results = []
    for filepath in csv_files:
        df = pd.read_csv(filepath, low_memory=False)
        df["plant"] = plant  # Add the 'plant' column
        results.append(df)
    df = pd.concat(results, ignore_index=True)
    return df

<IPython.core.display.Javascript object>

### Renaming the Columns

In [8]:
def preprocess_rename_columns(df):
    df.columns = pd.MultiIndex.from_tuples(
        list(
            {
                col: (col[0], "") if "Unnamed" in col[1] else col for col in df.columns
            }.values()
        )
    )
    return df

<IPython.core.display.Javascript object>

### Changing the order_of columns Columns

In [9]:
def preprocess_change_columns_order(df, column, pos):
    # Get the list of columns
    cols = df.columns.tolist()
    # Remove the specified column
    cols.remove(column)
    # Insert the column at the desired position
    cols.insert(pos, column)
    # Reorder the DataFrame columns
    return df[cols]

<IPython.core.display.Javascript object>

In [10]:
def read_csv_files_path(csv_files_path_dict, path, plant):
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    csv_files_path_dict[plant] = csv_files
    return csv_files_path_dict

<IPython.core.display.Javascript object>

# Reading The files

In [11]:
csv_files_path_grouped = dict()
csv_files_path_full = dict()

<IPython.core.display.Javascript object>

### 203

In [12]:
plants_203 = ["c"]

for plant in plants_203:
    path_grouped = f"/home/peressim/projects/ccs28-ml-modelling/reports/results/local_models/203/{plant}/grouped/"
    path_full = f"/home/peressim/projects/ccs28-ml-modelling/reports/results/local_models/203/{plant}/full/"
    csv_files_path_full = read_csv_files_path(csv_files_path_full, path_full, plant)
    csv_files_path_grouped = read_csv_files_path(
        csv_files_path_grouped, path_grouped, plant
    )

<IPython.core.display.Javascript object>

### Reading all data

#### Full

In [13]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_full.items():
    df = read_csv_files_full(csv_files, plant)
    df["plant"] = plant
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [14]:
df_full = pd.concat(dfs_plant.values(), ignore_index=True)

<IPython.core.display.Javascript object>

In [15]:
df_full.shape

(8684, 23)

<IPython.core.display.Javascript object>

In [16]:
filter_df = df_full[df_full["R2 Test"] < 0].reset_index(drop=True)

filter_df["CP"] = (
    filter_df["Company"].astype(str) + "_" + filter_df["plant"].astype(str)
)

filter_df["CP"].unique()

print(filter_df.shape)

(555, 24)


<IPython.core.display.Javascript object>

In [17]:
patterns = [
    "Chemical + Mineralogical + Feature Engineering",
    "Chemical + Mineralogical + CS7 + One-Hot",
    "Chemical + Mineralogical + Physical + One-Hot",
    "Chemical + Mineralogical + CS3 + One-Hot",
    "Chemical + Mineralogical + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + One-Hot",
    "Chemical + Mineralogical + CS1 + One-Hot",
    "Chemical + Mineralogical + CS2 + One-Hot",
    "Chemical + Feature Engineering",
]

FEATURES_TO_REPLACE_1 = {
    "Chemical + Mineralogical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS2": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS7": "Chemical + Mineralogical + Early CS",
}

FEATURES_TO_REPLACE_2 = {
    "Chemical": "Chemical + Mineralogical",
    "Chemical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Mineralogical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Physical": "Chemical + Mineralogical + Physical",
}

df_full = df_full[~df_full["Features"].apply(lambda x: x in patterns)].reset_index(
    drop=True
)

df_full = df_full[~df_full["Cross Validation"].eq("Out of time Split")].reset_index(
    drop=True
)

# removing afterwards
df_full = df_full[~df_full["Cross Validation"].eq("Repeated KFold")].reset_index(
    drop=True
)

df_full["Features_bkp"] = df_full["Features"].copy()
df_full["Features"] = df_full["Features"].replace(FEATURES_TO_REPLACE_1)

df_full["Features"] = df_full["Features"].replace(FEATURES_TO_REPLACE_2)


# removing afterwards
df_full = df_full[
    ~df_full["Features"].eq("Chemical + Mineralogical + Early CS")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [18]:
df_full["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
df_full["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [20]:
df_full["Features_bkp"].unique()

array(['Chemical + Properties CS Less', 'Chemical + Physical', 'Chemical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [21]:
df_full.shape

(1515, 24)

<IPython.core.display.Javascript object>

In [22]:
df_full[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
RMSE Test,1515.0,2.89,2.32,0.45,1.75,2.36,3.25,28.17
MAE Test,1515.0,1.98,1.21,0.38,1.31,1.71,2.28,12.46
MAPE Test,1515.0,0.05,0.03,0.01,0.03,0.04,0.05,0.30
R2 Test,1515.0,0.61,1.38,-21.80,0.70,0.84,0.91,0.99


<IPython.core.display.Javascript object>

In [23]:
df_full[df_full["R2 Test"] < 0][["Company", "plant"]]["plant"].nunique()

1

<IPython.core.display.Javascript object>

In [24]:
df_full[df_full["R2 Test"].eq(df_full["R2 Test"].min())]

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,RMSE Train,MAE Train,MAPE Train,R2 Train,RMSE Test,MAE Test,MAPE Test,R2 Test,plant,Features_bkp
439,Local Model,203,C,Chemical + Mineralogical + Physical,"(2829, 19)",14.0,BidirectionalGRU,NaN,Standard Scaler,NaN,...,0.910095,0.721416,0.016419,0.976681,28.173882,6.014505,0.129485,-21.801437,c,Chemical + Physical
444,Local Model,203,C,Chemical + Mineralogical + Physical,"(2829, 19)",14.0,BidirectionalGRU,NaN,Standard Scaler,NaN,...,0.910095,0.721416,0.016419,0.976681,28.173882,6.014505,0.129485,-21.801437,c,Chemical + Physical
449,Local Model,203,C,Chemical + Mineralogical + Physical,"(2829, 19)",14.0,BidirectionalGRU,NaN,Standard Scaler,NaN,...,0.910095,0.721416,0.016419,0.976681,28.173882,6.014505,0.129485,-21.801437,c,Chemical + Physical


<IPython.core.display.Javascript object>

In [25]:
df_full[(df_full["R2 Test"].eq(df_full["R2 Test"].min()))]["Cross Validation"]

439    Time Series Split
444    Time Series Split
449    Time Series Split
Name: Cross Validation, dtype: object

<IPython.core.display.Javascript object>

In [26]:
filter_df = df_full[df_full["R2 Test"] < 0].reset_index(drop=True)

print(filter_df.shape)

(90, 24)


<IPython.core.display.Javascript object>

In [27]:
df_full.shape

(1515, 24)

<IPython.core.display.Javascript object>

In [28]:
df_full["plant"].unique()

array(['c'], dtype=object)

<IPython.core.display.Javascript object>

In [29]:
df_full_203 = df_full[df_full["plant"].isin(["partner_i", "partner_ii", "partner_iv"])].reset_index(
    drop=True
)
df_full_203 = df_full[~df_full["plant"].isin(["partner_i", "partner_ii", "partner_iv"])].reset_index(
    drop=True
)

<IPython.core.display.Javascript object>

In [30]:
df_full.shape

(1515, 24)

<IPython.core.display.Javascript object>

In [31]:
df_full_203.shape

(1515, 24)

<IPython.core.display.Javascript object>

In [32]:
df_full_203.shape

(1515, 24)

<IPython.core.display.Javascript object>

In [33]:
df_full_203[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
RMSE Test,1515.0,2.89,2.32,0.45,1.75,2.36,3.25,28.17
MAE Test,1515.0,1.98,1.21,0.38,1.31,1.71,2.28,12.46
MAPE Test,1515.0,0.05,0.03,0.01,0.03,0.04,0.05,0.30
R2 Test,1515.0,0.61,1.38,-21.80,0.70,0.84,0.91,0.99


<IPython.core.display.Javascript object>

In [34]:
df_full_203[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
RMSE Test,1515.0,2.89,2.32,0.45,1.75,2.36,3.25,28.17
MAE Test,1515.0,1.98,1.21,0.38,1.31,1.71,2.28,12.46
MAPE Test,1515.0,0.05,0.03,0.01,0.03,0.04,0.05,0.30
R2 Test,1515.0,0.61,1.38,-21.80,0.70,0.84,0.91,0.99


<IPython.core.display.Javascript object>

# Analysis (full results)

In [35]:
df_full_203 = df_full[~df_full["plant"].isin(["partner_i", "partner_ii", "partner_iv"])].reset_index(
    drop=True
)

<IPython.core.display.Javascript object>

## 203

In [36]:
df_full_203.shape

(1515, 24)

<IPython.core.display.Javascript object>

In [37]:
df_full_203["Company"].unique()

array([203])

<IPython.core.display.Javascript object>

### Variable Grouping: CM

In [38]:
df_full_203["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [39]:
df_results_full_cm = (
    df_full_203[df_full_203["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [40]:
df_results_full_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [41]:
df_results_full_cm.shape

(520, 24)

<IPython.core.display.Javascript object>

#### K-Fold

In [42]:
df_results_full_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [43]:
df_results_full_cm_kf = (
    df_results_full_cm[df_results_full_cm["Cross Validation"].eq("Repeated KFold")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [44]:
df_results_full_cm_kf["Cross Validation"].unique()

array([], dtype=object)

<IPython.core.display.Javascript object>

In [45]:
df_results_full_cm_kf.shape

(0, 24)

<IPython.core.display.Javascript object>

In [46]:
df_results_full_cm_kf.groupby(["Company", "Model", "Timesteps"], dropna=False)[
    "RMSE Test"
].mean().shape

(0,)

<IPython.core.display.Javascript object>

### Blocking time series

In [47]:
df_results_full_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [48]:
df_results_full_cm_btss = (
    df_results_full_cm[
        df_results_full_cm["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [49]:
df_results_full_cm_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [50]:
df_results_full_cm_btss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [51]:
df_results_full_cm_btss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Time Series Split

In [52]:
df_results_full_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [53]:
df_results_full_cm_tss = (
    df_results_full_cm[df_results_full_cm["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [54]:
df_results_full_cm_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [55]:
df_results_full_cm_tss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [56]:
df_results_full_cm_tss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Variable Grouping: CM-P

In [57]:
df_full_203["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [58]:
df_results_full_cm_p = (
    df_full_203[
        df_full_203["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [59]:
df_results_full_cm_p["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [60]:
df_results_full_cm_p.shape

(520, 24)

<IPython.core.display.Javascript object>

### K-Fold

In [61]:
# df_results_full_cm_p["Cross Validation"].unique()

<IPython.core.display.Javascript object>

In [62]:
# df_results_full_cm_p_kf = (
#     df_results_full_cm_p[df_results_full_cm_p["Cross Validation"].eq("Repeated KFold")]
#     .copy()
#     .reset_index(drop=True)
# )

<IPython.core.display.Javascript object>

In [63]:
# df_results_full_cm_p_kf["Cross Validation"].unique()

<IPython.core.display.Javascript object>

In [64]:
# df_results_full_cm_p_kf.shape

<IPython.core.display.Javascript object>

In [65]:
# df_results_full_cm_kf.groupby(["Company", "Model", "Timesteps"], dropna=False)[
#     "RMSE Test"
# ].mean().shape

<IPython.core.display.Javascript object>

### Blocking time series

In [66]:
df_results_full_cm_p["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [67]:
df_results_full_cm_p_btss = (
    df_results_full_cm_p[
        df_results_full_cm_p["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [68]:
df_results_full_cm_p_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [69]:
df_results_full_cm_p_btss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [70]:
df_results_full_cm_p_btss.shape, df_results_full_cm_btss.shape

((260, 24), (260, 24))

<IPython.core.display.Javascript object>

In [71]:
df_results_full_cm_p_btss.groupby(
    ["Company", "Cross Validation", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

In [72]:
df_results_full_cm_btss.shape, df_results_full_cm_p_btss.shape

df1 = df_results_full_cm_btss
df2 = df_results_full_cm_p_btss

grouping_columns = ["Company", "Plant", "Model", "Timesteps"]

<IPython.core.display.Javascript object>

In [73]:
# Group by and get sizes for both datasets
group_sizes_df1 = df1.groupby(grouping_columns, dropna=False).size()
group_sizes_df2 = df2.groupby(grouping_columns, dropna=False).size()

# Compare group sizes directly
group_size_diff = group_sizes_df1.compare(group_sizes_df2)
print("Differences in group sizes:")
print(group_size_diff)

Differences in group sizes:
Empty DataFrame
Columns: [self, other]
Index: []


<IPython.core.display.Javascript object>

In [74]:
# Group by and get sizes for both datasets
group_sizes_df1 = df1.groupby(grouping_columns, dropna=False).size()
group_sizes_df2 = df2.groupby(grouping_columns, dropna=False).size()

# Compare group sizes directly
group_size_diff = group_sizes_df2.compare(group_sizes_df1)
print("Differences in group sizes:")
print(group_size_diff)

Differences in group sizes:
Empty DataFrame
Columns: [self, other]
Index: []


<IPython.core.display.Javascript object>

### Time Series Split

In [75]:
df_results_full_cm_p["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [76]:
df_results_full_cm_p_tss = (
    df_results_full_cm_p[
        df_results_full_cm_p["Cross Validation"].eq("Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [77]:
df_results_full_cm_p_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [78]:
df_results_full_cm_p_tss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [79]:
df_results_full_cm_p_tss.groupby(
    ["Company", "Model", "Plant", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Variable Grouping: CM-P-CS

In [80]:
df_full_203["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [81]:
df_results_full_cm_p_cs = (
    df_full_203[df_full_203["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [82]:
df_results_full_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [83]:
df_results_full_cm_p_cs.shape

(475, 24)

<IPython.core.display.Javascript object>

### K-Fold

In [84]:
df_results_full_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [85]:
df_results_full_cm_p_cs_kf = (
    df_results_full_cm_p_cs[
        df_results_full_cm_p_cs["Cross Validation"].eq("Repeated KFold")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [86]:
df_results_full_cm_p_cs_kf["Cross Validation"].unique()

array([], dtype=object)

<IPython.core.display.Javascript object>

In [87]:
df_results_full_cm_p_cs_kf.shape

(0, 24)

<IPython.core.display.Javascript object>

In [88]:
df_results_full_cm_p_cs_kf.groupby(["Company", "Model", "Timesteps"], dropna=False)[
    "RMSE Test"
].mean().shape

(0,)

<IPython.core.display.Javascript object>

### Blocking time series

In [89]:
df_results_full_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [90]:
df_results_full_cm_p_cs_btss = (
    df_results_full_cm_p_cs[
        df_results_full_cm_p_cs["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [91]:
df_results_full_cm_p_cs_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [92]:
df_results_full_cm_p_cs_btss.shape

(260, 24)

<IPython.core.display.Javascript object>

In [93]:
df_results_full_cm_p_cs_btss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(20,)

<IPython.core.display.Javascript object>

### Time Series Split

In [94]:
df_results_full_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [95]:
df_results_full_cm_p_cs_tss = (
    df_results_full_cm_p_cs[
        df_results_full_cm_p_cs["Cross Validation"].eq("Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [96]:
df_results_full_cm_p_cs_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [97]:
df_results_full_cm_p_cs_tss.shape

(215, 24)

<IPython.core.display.Javascript object>

In [98]:
df_results_full_cm_p_cs_tss.groupby(
    ["Company", "Plant", "Model", "Timesteps"], dropna=False
)["RMSE Test"].mean().shape

(17,)

<IPython.core.display.Javascript object>

# Grouped Dataset

In [99]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_grouped.items():
    df = read_csv_files_grouped(csv_files, plant)
    df["plant"] = plant
    df = preprocess_rename_columns(df)
    df = preprocess_change_columns_order(df, column, pos)
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [100]:
df = pd.concat(dfs_plant)
df_copy = df.copy()
df_copy = (
    df_copy.reset_index(level=0)
    .rename({"level_0": "Plant"}, axis=1)
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [101]:
df_copy.shape

(797, 26)

<IPython.core.display.Javascript object>

## Preprocessing steps

In [102]:
patterns = [
    "Chemical + Mineralogical + Feature Engineering",
    "Chemical + Mineralogical + CS7 + One-Hot",
    "Chemical + Mineralogical + Physical + One-Hot",
    "Chemical + Mineralogical + CS3 + One-Hot",
    "Chemical + Mineralogical + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + One-Hot",
    "Chemical + Mineralogical + CS1 + One-Hot",
    "Chemical + Mineralogical + CS2 + One-Hot",
    "Chemical + Feature Engineering",
]

replace_dict = {
    "Chemical + Mineralogical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS2": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS7": "Chemical + Mineralogical + Early CS",
}

replace_dict_2 = {
    "Chemical": "Chemical + Mineralogical",
    "Chemical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Mineralogical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Physical": "Chemical + Mineralogical + Physical",
}

df_copy = df_copy[~df_copy["Cross Validation"].eq("Out of time Split")].reset_index(
    drop=True
)

# Removing afterwards
df_copy = df_copy[~df_copy["Cross Validation"].eq("Repeated KFold")].reset_index(
    drop=True
)

df_copy = df_copy[~df_copy["Features"].apply(lambda x: x in patterns)].reset_index(
    drop=True
)
df_copy["Features_bkp"] = df_copy["Features"].copy()
df_copy["Features"] = df_copy["Features"].replace(replace_dict)

df_copy["Features"] = df_copy["Features"].replace(replace_dict_2)

# Removing afterwards
df_copy = df_copy[
    ~df_copy["Features"].eq("Chemical + Mineralogical + Early CS")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [103]:
df_copy[("MAPE Train", "mean")] = df_copy[("MAPE Train", "mean")] * 100
df_copy[("MAPE Train", "std")] = df_copy[("MAPE Train", "std")] * 100
df_copy[("MAPE Test", "mean")] = df_copy[("MAPE Test", "mean")] * 100
df_copy[("MAPE Test", "std")] = df_copy[("MAPE Test", "std")] * 100

df_copy["Model_bkp"] = df_copy["Model"].copy()
df_copy["Model_bkp_2"] = df_copy["Model"] + df_copy["Timesteps"].apply(
    lambda x: "" if pd.isna(x) else "_" + str(x)
)
df_copy["Model"] = df_copy["Model"].replace(
    {
        "MLP": "Neural Networks",
        "LSTM": "Neural Networks",
        "GRU": "Neural Networks",
        "BidirectionalLSTM": "Neural Networks",
        "BidirectionalGRU": "Neural Networks",
        "Transformer": "Neural Networks",
        "Decision Tree": "Trees",
        "Random Forest": "Trees",
        "XGBoost": "Trees",
    }
)

<IPython.core.display.Javascript object>

In [104]:
df_copy["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [105]:
df_copy["Features"].unique()

array(['Chemical + Mineralogical + Physical', 'Chemical + Mineralogical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [106]:
df_copy["Model"].unique()

array(['Trees', 'Neural Networks', 'Linear Regression'], dtype=object)

<IPython.core.display.Javascript object>

In [107]:
df_copy["Model_bkp"].unique()

array(['Random Forest', 'Transformer', 'BidirectionalGRU', 'GRU', 'MLP',
       'Decision Tree', 'BidirectionalLSTM', 'XGBoost',
       'Linear Regression', 'LSTM'], dtype=object)

<IPython.core.display.Javascript object>

In [108]:
df_copy["Model_bkp_2"].unique()

array(['Random Forest', 'Transformer_1.0', 'Transformer_7.0',
       'Transformer_14.0', 'BidirectionalGRU_1.0', 'BidirectionalGRU_7.0',
       'BidirectionalGRU_14.0', 'GRU_1.0', 'GRU_7.0', 'GRU_14.0', 'MLP',
       'Decision Tree', 'BidirectionalLSTM_1.0', 'BidirectionalLSTM_7.0',
       'BidirectionalLSTM_14.0', 'XGBoost', 'Linear Regression',
       'LSTM_1.0', 'LSTM_7.0', 'LSTM_14.0'], dtype=object)

<IPython.core.display.Javascript object>

In [109]:
df_copy.shape

(117, 29)

<IPython.core.display.Javascript object>

In [110]:
df_copy_grouped = df_copy.copy()

<IPython.core.display.Javascript object>

In [111]:
df_copy[
    [
        ("RMSE Test", "mean"),
        ("MAE Test", "mean"),
        ("MAPE Test", "mean"),
        ("R2 Test", "mean"),
    ]
].describe().round(2).T

,,count,mean,std,min,25%,50%,75%,max
RMSE Test,mean,117.0,2.75,1.14,0.88,1.98,2.60,3.29,7.75
MAE Test,mean,117.0,1.90,0.61,0.66,1.49,1.90,2.31,3.78
MAPE Test,mean,117.0,4.45,1.42,1.54,3.47,4.41,5.42,9.24
R2 Test,mean,117.0,0.64,0.58,-3.75,0.64,0.79,0.87,0.98


<IPython.core.display.Javascript object>

In [112]:
(df_copy[("R2 Test", "mean")] < 0).sum()

5

<IPython.core.display.Javascript object>

In [113]:
(df_copy[("MAPE Test", "mean")] > 100).sum()

0

<IPython.core.display.Javascript object>

In [114]:
(df_copy[("RMSE Test", "mean")] > 100).sum()

0

<IPython.core.display.Javascript object>

In [115]:
(df_copy[("MAE Test", "mean")] > 100).sum()

0

<IPython.core.display.Javascript object>

In [116]:
(
    (df_copy[("R2 Test", "mean")] < 0)
    | (df_copy[("MAPE Test", "mean")] > 100)
    | (df_copy[("RMSE Test", "mean")] > 100)
    | (df_copy[("MAE Test", "mean")] > 100)
).sum()

5

<IPython.core.display.Javascript object>

In [117]:
df_copy_2 = df_copy.copy()

<IPython.core.display.Javascript object>

In [118]:
df_copy = df_copy[
    ~(
        (df_copy[("R2 Test", "mean")] < 0)
        | (df_copy[("MAPE Test", "mean")] > 100)
        | (df_copy[("RMSE Test", "mean")] > 100)
        | (df_copy[("MAE Test", "mean")] > 100)
    )
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [119]:
df_copy[
    [
        ("RMSE Test", "mean"),
        ("MAE Test", "mean"),
        ("MAPE Test", "mean"),
        ("R2 Test", "mean"),
    ]
].describe().round(2).T

,,count,mean,std,min,25%,50%,75%,max
RMSE Test,mean,112.0,2.60,0.86,0.88,1.97,2.51,3.21,4.58
MAE Test,mean,112.0,1.86,0.58,0.66,1.46,1.85,2.27,3.46
MAPE Test,mean,112.0,4.36,1.35,1.54,3.44,4.36,5.31,8.24
R2 Test,mean,112.0,0.74,0.19,0.09,0.66,0.79,0.88,0.98


<IPython.core.display.Javascript object>

In [120]:
em_copy = df_copy[
    [
        ("RMSE Test", "mean"),
        ("MAE Test", "mean"),
        ("MAPE Test", "mean"),
        ("R2 Test", "mean"),
    ]
].copy()

em_copy.columns = ["RMSE", "MAE", "MAPE", "R2"]
em_copy.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
RMSE,112.0,2.60,0.86,0.88,1.97,2.51,3.21,4.58
MAE,112.0,1.86,0.58,0.66,1.46,1.85,2.27,3.46
MAPE,112.0,4.36,1.35,1.54,3.44,4.36,5.31,8.24
R2,112.0,0.74,0.19,0.09,0.66,0.79,0.88,0.98


<IPython.core.display.Javascript object>

In [121]:
df_copy[
    [
        ("RMSE Test", "std"),
        ("MAE Test", "std"),
        ("MAPE Test", "std"),
        ("R2 Test", "std"),
    ]
].describe().round(2).T

,,count,mean,std,min,25%,50%,75%,max
RMSE Test,std,112.0,1.01,0.74,0.06,0.49,0.81,1.32,4.12
MAE Test,std,112.0,0.67,0.60,0.06,0.34,0.51,0.79,4.07
MAPE Test,std,112.0,1.58,1.47,0.15,0.73,1.16,1.78,9.73
R2 Test,std,112.0,0.21,0.26,0.00,0.06,0.13,0.27,1.60


<IPython.core.display.Javascript object>

In [122]:
df_copy.shape

(112, 29)

<IPython.core.display.Javascript object>

In [123]:
df_copy["plant"].unique()

array(['c'], dtype=object)

<IPython.core.display.Javascript object>

In [124]:
df_203_groupd = df_copy_grouped[
    ~df_copy_grouped["plant"].isin(["partner_i", "partner_ii", "partner_iv"])
]

<IPython.core.display.Javascript object>

In [125]:
df_203_groupd.shape

(117, 29)

<IPython.core.display.Javascript object>

In [126]:
df_full.shape

(1515, 24)

<IPython.core.display.Javascript object>

In [127]:
cols_groupby = [
    # "Category",
    # "Company",
    "plant",
    # "Data Shape",
    "Timesteps",
    "Features_bkp",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

df_results_to_save = (
    df_full.groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

<IPython.core.display.Javascript object>

In [128]:
df_results_to_save.shape

(117, 22)

<IPython.core.display.Javascript object>

In [129]:
df_copy_grouped.shape

(117, 29)

<IPython.core.display.Javascript object>

# Topological Analysis - NEW

## Definitions

In [130]:
# Function to compare two models strictly based on means only
def dominates(model1, model2):
    """Return True if model1 strictly dominates model2, False otherwise."""
    # Check if model1 is at least as good as model2 in all metrics
    at_least_as_good = (
        model1["RMSE_mean"] <= model2["RMSE_mean"]
        and model1["MAE_mean"] <= model2["MAE_mean"]
        and model1["MAPE_mean"] <= model2["MAPE_mean"]
        and model1["R2_mean"] >= model2["R2_mean"]
    )

    # Check if model1 is strictly better in at least one metric
    strictly_better_in_one = (
        model1["RMSE_mean"] < model2["RMSE_mean"]
        or model1["MAE_mean"] < model2["MAE_mean"]
        or model1["MAPE_mean"] < model2["MAPE_mean"]
        or model1["R2_mean"] > model2["R2_mean"]
    )

    # Model1 strictly dominates Model2 if it is at least as good in all metrics and
    # strictly better in at least one metric
    return at_least_as_good and strictly_better_in_one

<IPython.core.display.Javascript object>

In [131]:
def get_dominance_matrix_and_graph(df):
    n = len(df)
    dominance_matrix = np.zeros((n, n), dtype=bool)

    for i in range(n):
        for j in range(n):
            if i != j:
                dominance_matrix[i, j] = dominates(df.iloc[i], df.iloc[j])

    # Create the dominance graph
    dominance_graph = {i: [] for i in range(n)}
    for i in range(n):
        for j in range(n):
            if dominance_matrix[i, j]:
                dominance_graph[i].append(j)

    return dominance_matrix, dominance_graph

<IPython.core.display.Javascript object>

In [132]:
# Topological Sorting
def topological_sort(graph):
    """Perform topological sorting on the given graph."""
    in_degree = {u: 0 for u in graph}
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1

    queue = deque([u for u in graph if in_degree[u] == 0])
    topo_order = []

    while queue:
        u = queue.popleft()
        topo_order.append(u)

        for v in graph[u]:
            in_degree[v] -= 1
            if in_degree[v] == 0:
                queue.append(v)

    return topo_order

<IPython.core.display.Javascript object>

In [133]:
# Function to find the top models in each group
def find_top_models(group):
    max_net_dominance = group["Net_Dominance"].max()
    top_models = group[group["Net_Dominance"] == max_net_dominance]
    return top_models

<IPython.core.display.Javascript object>

### Dominance Analysis and SCPM

In [134]:
def compute_scpm(df):
    scaler = StandardScaler()
    standardized_metrics = scaler.fit_transform(
        df[["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"]]
    )

    # Creating a new dataframe with standardized values
    standardized_df = pd.DataFrame(
        standardized_metrics,
        columns=["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"],
    )

    # Summing all metrics and subtracting R2_mean
    standardized_df["Result"] = (
        standardized_df[["RMSE_mean", "MAE_mean", "MAPE_mean"]].sum(axis=1)
        - standardized_df["R2_mean"]
    )

    df["SCPM"] = standardized_df["Result"]
    return df

<IPython.core.display.Javascript object>

In [135]:
def make_dominance_analysis(df):
    # Compute dominance matrix and graph
    dominance_matrix, dominance_graph = get_dominance_matrix_and_graph(df)

    # Get the topological order
    topo_order = topological_sort(dominance_graph)
    df = df.copy()
    n = len(topo_order)

    # Identify Dominant Models
    dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if not np.any(dominance_matrix[:, i])
    ]

    # Identify Non-Dominant Models
    non_dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i])
    ]

    # Identify Models that are both dominated and dominate at least one
    intermediate_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i]) and np.any(dominance_matrix[i, :])
    ]

    # Add count of times each model is dominated by others
    df["Dominated_Count"] = np.sum(dominance_matrix, axis=0)

    # Classify models
    df["Classification"] = "intermediate_model"  # default to intermediate_model
    df.loc[
        ~np.any(dominance_matrix, axis=1) & np.any(dominance_matrix, axis=0),
        "Classification",
    ] = "non_dominant_model"
    df.loc[~np.any(dominance_matrix, axis=0), "Classification"] = "dominant_model"

    # Map the topological order to the model names and get dominance counts
    sorted_models = [df["Model"].iloc[i] for i in topo_order]

    df_sorted = df.iloc[topo_order].copy()
    df_sorted["Dominates_Count"] = dominance_matrix.sum(axis=1)[topo_order]
    df_sorted_topo = df_sorted.copy()

    df_sorted = df_sorted.sort_values(by="Dominates_Count", ascending=False)
    df_sorted_count = df_sorted.copy()

    return {
        "dominance_matrix": dominance_matrix,
        "dominance_graph": dominance_graph,
        "df_sorted_topo": df_sorted_topo,
        "df_sorted_count": df_sorted_count,
    }

<IPython.core.display.Javascript object>

# Analysis

## 203

In [136]:
# Filter only 203 project
df_203_grouped = df_copy_grouped[
    ~df_copy_grouped["plant"].isin(["partner_i", "partner_ii", "partner_iv"])
]

<IPython.core.display.Javascript object>

In [137]:
df_results = df_203_grouped[
    [
        ("Company", ""),
        ("plant", ""),
        ("Cross Validation", ""),
        ("Features", ""),
        ('Features_bkp', ''),
        ("Model", ""),
        ("Model_bkp", ""),         
        ('Model_bkp_2',''),
        ("Timesteps", ""),
        ("RMSE Test", "mean"),
        ("MAE Test", "mean"),
        ("MAPE Test", "mean"),
        ("R2 Test", "mean"),
        ("RMSE Test", "std"),
        ("MAE Test", "std"),
        ("MAPE Test", "std"),
        ("R2 Test", "std"),
    ]
].copy()
df_results = df_results.reset_index(drop=True)

# Rename columns
new_column_names = [
    "Company",
    "Plant",
    "Cross Validation",
    "Features",
    "Features_bkp",
    "Model",
    "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "RMSE_std",
    "MAE_std",
    "MAPE_std",
    "R2_std",
]

df_results.columns = new_column_names


<IPython.core.display.Javascript object>

In [138]:
df_results.shape

(117, 17)

<IPython.core.display.Javascript object>

In [139]:
df_results["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

## Variable Grouping: CM

In [140]:
df_results["Features"].unique()

array(['Chemical + Mineralogical + Physical', 'Chemical + Mineralogical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [141]:
df_results_cm = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [142]:
df_results_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [143]:
df_results_cm.shape

(40, 17)

<IPython.core.display.Javascript object>

### Blocking time series

In [144]:
df_results_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [145]:
df_results_cm_btss = (
    df_results_cm[df_results_cm["Cross Validation"].eq("Blocking Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [146]:
df_results_cm_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [147]:
df_results_cm_btss.shape

(20, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [148]:
df_results_cm_btss = compute_scpm(df_results_cm_btss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [149]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_btss)
dominance_matrix_cm_btss = dominance_dict["dominance_matrix"]
dominance_graph_cm_btss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [150]:
dominance_matrix_cm_btss.shape, len(dominance_graph_cm_btss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [151]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    18
dominant_model         1
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [152]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/203/203_c_dominance_analysis_cm_btss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [153]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,203,c,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,Transformer,Transformer_1.0,1.0,1.537175,...,2.488966,0.928655,0.468216,0.216604,0.541965,0.035101,-5.021284,0,dominant_model,19
13,203,c,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Trees,Random Forest,Random Forest,NaN,1.690821,...,2.506176,0.911171,0.604527,0.347460,0.740677,0.052418,-4.739528,1,intermediate_model,17
16,203,c,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Trees,Decision Tree,Decision Tree,NaN,1.818787,...,2.518619,0.897298,0.587358,0.188049,0.422755,0.056023,-4.547784,1,intermediate_model,17
1,203,c,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,Transformer,Transformer_7.0,7.0,1.885277,...,3.305065,0.855936,1.274987,0.663642,2.069809,0.190961,-3.624045,3,intermediate_model,13
3,203,c,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,BidirectionalGRU,BidirectionalGRU_1.0,1.0,1.954474,...,3.416799,0.885433,0.528196,0.369395,0.897439,0.045507,-3.375192,3,intermediate_model,14


<IPython.core.display.Javascript object>

In [154]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [155]:
# Top 5
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[0:5]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,Transformer_1.0,1.54,0.47,1.06,0.22,2.49,0.54,0.93,0.04,-5.02,0,dominant_model,19,c
13,Random Forest,1.69,0.60,1.07,0.35,2.51,0.74,0.91,0.05,-4.74,1,intermediate_model,17,c
16,Decision Tree,1.82,0.59,1.06,0.19,2.52,0.42,0.90,0.06,-4.55,1,intermediate_model,17,c
3,BidirectionalGRU_1.0,1.95,0.53,1.46,0.37,3.42,0.90,0.89,0.05,-3.38,3,intermediate_model,14,c
1,Transformer_7.0,1.89,1.27,1.29,0.66,3.31,2.07,0.86,0.19,-3.62,3,intermediate_model,13,c


<IPython.core.display.Javascript object>

In [156]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [157]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
13,Random Forest,1.69,0.60,1.07,0.35,2.51,0.74,0.91,0.05,-4.74,1,intermediate_model,17,c
16,Decision Tree,1.82,0.59,1.06,0.19,2.52,0.42,0.90,0.06,-4.55,1,intermediate_model,17,c
3,BidirectionalGRU_1.0,1.95,0.53,1.46,0.37,3.42,0.90,0.89,0.05,-3.38,3,intermediate_model,14,c


<IPython.core.display.Javascript object>

In [158]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(18, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [159]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:2]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
10,BidirectionalLSTM_7.0,4.46,4.72,3.78,4.34,9.24,10.6,-0.16,2.05,9.34,19,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [160]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [161]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [162]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [163]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [164]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [165]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [166]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression intermediate_model      100.00
Neural Networks   intermediate_model       87.50
                  dominant_model            6.25
                  non_dominant_model        6.25
Trees             intermediate_model      100.00

<IPython.core.display.Javascript object>

In [167]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                 5.0            10.12   2.67
                std                  NaN             5.86   2.89
                min                  5.0             0.00   1.00
                25%                  5.0             4.00   1.00
                50%                  5.0            10.50   1.00
                75%                  5.0            13.75   3.50
                max                  5.0            19.00   6.00
Dominates_Count count                1.0            16.00   3.00
                mean                11.0             7.44  15.00
                std                  NaN             5.48   3.46
                min                 11.0             0.00  11.00
                25%                 11.0             3.50  14.00
                50%                 11.0             7.00  17.00
                75%                 11.0            11.50  17.00
                max                 11.0            19.00  17.00

<IPython.core.display.Javascript object>

In [168]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [169]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,intermediate_model,1
Neural Networks,16,3,intermediate_model,14
Trees,3,1,intermediate_model,3


<IPython.core.display.Javascript object>

In [170]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  intermediate_model     1
Neural Networks    intermediate_model    14
                   dominant_model         1
                   non_dominant_model     1
Trees              intermediate_model     3
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [171]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [172]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [173]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,11.0000,11.0,5,11,0.55
Neural Networks,16,7.4375,7.0,162,119,5.95
Trees,3,15.0000,17.0,8,45,2.25


<IPython.core.display.Javascript object>

In [174]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  2.150860       NaN  2.150860  2.150860  2.150860   
Neural Networks        16.0  2.881803  0.832540  1.537175  2.074038  3.038227   
Trees                   3.0  1.897274  0.254925  1.690821  1.754804  1.818787   

                                      RMSE_std            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  2.150860  2.150860      1.0  0.621980  ...            5.00   
Neural Networks    3.458874  4.459451     16.0  1.831552  ...           13.75   
Trees              2.000500  2.182213      3.0  0.620186  ...            3.50   

                        Dominates_Count                                       \
                    max           count     mean       std   min   25%   50%   
Model                                                                          
Linear Regression   5.0             1.0  11.0000       NaN  11.0  11.0  11.0   
Neural Networks    19.0            16.0   7.4375  5.476845   0.0   3.5   7.0   
Trees               6.0             3.0  15.0000  3.464102  11.0  14.0  17.0   

                               
                    75%   max  
Model                          
Linear Regression  11.0  11.0  
Neural Networks    11.5  19.0  
Trees              17.0  17.0  

[3 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [175]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [176]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [177]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [178]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
0,Transformer_1.0,1.54,0.47,1.06,0.22,2.49,0.54,0.93,0.04,-5.02,0,19,dominant_model,c,True
13,Random Forest,1.69,0.60,1.07,0.35,2.51,0.74,0.91,0.05,-4.74,1,17,intermediate_model,c,True
16,Decision Tree,1.82,0.59,1.06,0.19,2.52,0.42,0.90,0.06,-4.55,1,17,intermediate_model,c,True
1,Transformer_7.0,1.89,1.27,1.29,0.66,3.31,2.07,0.86,0.19,-3.62,3,13,intermediate_model,c,True
3,BidirectionalGRU_1.0,1.95,0.53,1.46,0.37,3.42,0.90,0.89,0.05,-3.38,3,14,intermediate_model,c,True


<IPython.core.display.Javascript object>

### Time Series Split

In [179]:
df_results_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [180]:
df_results_cm_tss = (
    df_results_cm[df_results_cm["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [181]:
df_results_cm_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [182]:
df_results_cm_tss.shape

(20, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [183]:
df_results_cm_tss = compute_scpm(df_results_cm_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [184]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_tss)
dominance_matrix_cm_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [185]:
dominance_matrix_cm_tss.shape, len(dominance_graph_cm_tss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [186]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    16
dominant_model         3
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [187]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/203_c_dominance_analysis_cm_tss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [188]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
1,203,c,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,Transformer,Transformer_7.0,7.0,2.374921,...,4.081860,0.829885,0.446533,0.265232,0.617194,0.060037,-4.372614,0,dominant_model,16
13,203,c,Time Series Split,Chemical + Mineralogical,Chemical,Trees,Random Forest,Random Forest,NaN,2.469346,...,3.743004,0.808014,0.831938,0.538519,1.177852,0.120594,-4.813562,0,dominant_model,17
16,203,c,Time Series Split,Chemical + Mineralogical,Chemical,Trees,Decision Tree,Decision Tree,NaN,2.613836,...,3.471669,0.781673,0.997199,0.504591,1.067576,0.151227,-5.218253,0,dominant_model,15
0,203,c,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,Transformer,Transformer_1.0,1.0,2.517192,...,4.028560,0.805481,0.612866,0.336935,0.923056,0.100521,-4.277953,1,intermediate_model,16
18,203,c,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,LSTM,LSTM_7.0,7.0,2.604495,...,4.371307,0.798854,0.340278,0.297855,0.685576,0.053949,-3.185973,3,intermediate_model,14


<IPython.core.display.Javascript object>

In [189]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [190]:
# Top 5
df_sorted_topo_cols.iloc[0:5].sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
13,Random Forest,2.47,0.83,1.62,0.54,3.74,1.18,0.81,0.12,-4.81,0,dominant_model,17,c
1,Transformer_7.0,2.37,0.45,1.70,0.27,4.08,0.62,0.83,0.06,-4.37,0,dominant_model,16,c
0,Transformer_1.0,2.52,0.61,1.67,0.34,4.03,0.92,0.81,0.10,-4.28,1,intermediate_model,16,c
16,Decision Tree,2.61,1.00,1.50,0.50,3.47,1.07,0.78,0.15,-5.22,0,dominant_model,15,c
18,LSTM_7.0,2.60,0.34,1.90,0.30,4.37,0.69,0.80,0.05,-3.19,3,intermediate_model,14,c


<IPython.core.display.Javascript object>

In [191]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(3, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [192]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,Transformer_1.0,2.52,0.61,1.67,0.34,4.03,0.92,0.81,0.10,-4.28,1,intermediate_model,16,c
18,LSTM_7.0,2.60,0.34,1.90,0.30,4.37,0.69,0.80,0.05,-3.19,3,intermediate_model,14,c
12,XGBoost,2.74,0.83,2.06,0.69,4.90,1.51,0.77,0.13,-1.90,5,intermediate_model,11,c
2,Transformer_14.0,2.88,0.77,2.08,0.52,5.02,1.43,0.74,0.14,-1.47,6,intermediate_model,10,c


<IPython.core.display.Javascript object>

In [193]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(16, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [194]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
9,BidirectionalLSTM_1.0,6.278918,4.369569,2.978662,0.965418,6.645053,2.047407,-0.699758,2.293754,10.718797,19,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [195]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [196]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [197]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [198]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [199]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [200]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression intermediate_model      100.00
Neural Networks   intermediate_model       87.50
                  dominant_model            6.25
                  non_dominant_model        6.25
Trees             dominant_model           66.67
                  intermediate_model       33.33

<IPython.core.display.Javascript object>

In [201]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                 8.0             9.12   1.67
                std                  NaN             5.92   2.89
                min                  8.0             0.00   0.00
                25%                  8.0             4.75   0.00
                50%                  8.0             8.50   0.00
                75%                  8.0            13.50   2.50
                max                  8.0            19.00   5.00
Dominates_Count count                1.0            16.00   3.00
                mean                 4.0             7.00  14.33
                std                  NaN             5.20   3.06
                min                  4.0             0.00  11.00
                25%                  4.0             2.75  13.00
                50%                  4.0             6.50  15.00
                75%                  4.0            10.00  16.00
                max                  4.0            16.00  17.00

<IPython.core.display.Javascript object>

In [202]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [203]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,intermediate_model,1
Neural Networks,16,3,intermediate_model,14
Trees,3,2,dominant_model,2


<IPython.core.display.Javascript object>

In [204]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  intermediate_model     1
Neural Networks    intermediate_model    14
                   dominant_model         1
                   non_dominant_model     1
Trees              dominant_model         2
                   intermediate_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [205]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [206]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,4.00,4.0,8,4,0.20
Neural Networks,16,7.00,6.5,146,112,5.60
Trees,3,14.33,15.0,5,43,2.15


<IPython.core.display.Javascript object>

In [207]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  3.237811       NaN  3.237811  3.237811  3.237811   
Neural Networks        16.0  3.651869  0.990150  2.374921  3.000890  3.507452   
Trees                   3.0  2.608345  0.136337  2.469346  2.541591  2.613836   

                                      RMSE_std            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  3.237811  3.237811      1.0  0.768772  ...             8.0   
Neural Networks    4.118500  6.278918     16.0  1.375059  ...            13.5   
Trees              2.677844  2.741853      3.0  0.885118  ...             2.5   

                        Dominates_Count                                    \
                    max           count       mean       std   min    25%   
Model                                                                       
Linear Regression   8.0             1.0   4.000000       NaN   4.0   4.00   
Neural Networks    19.0            16.0   7.000000  5.202563   0.0   2.75   
Trees               5.0             3.0  14.333333  3.055050  11.0  13.00   

                                     
                    50%   75%   max  
Model                                
Linear Regression   4.0   4.0   4.0  
Neural Networks     6.5  10.0  16.0  
Trees              15.0  16.0  17.0  

[3 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [208]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [209]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [210]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [211]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
16,Decision Tree,2.61,1.00,1.50,0.50,3.47,1.07,0.78,0.15,-5.22,0,15,dominant_model,c,False
13,Random Forest,2.47,0.83,1.62,0.54,3.74,1.18,0.81,0.12,-4.81,0,17,dominant_model,c,True
1,Transformer_7.0,2.37,0.45,1.70,0.27,4.08,0.62,0.83,0.06,-4.37,0,16,dominant_model,c,False
0,Transformer_1.0,2.52,0.61,1.67,0.34,4.03,0.92,0.81,0.10,-4.28,1,16,intermediate_model,c,True
18,LSTM_7.0,2.60,0.34,1.90,0.30,4.37,0.69,0.80,0.05,-3.19,3,14,intermediate_model,c,True


<IPython.core.display.Javascript object>

### Variable Grouping: CM-P

In [212]:
df_results["Features"].unique()

array(['Chemical + Mineralogical + Physical', 'Chemical + Mineralogical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [213]:
df_results_cm_p = (
    df_results[
        df_results["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [214]:
df_results_cm_p["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [215]:
df_results_cm_p.shape

(40, 17)

<IPython.core.display.Javascript object>

### Blocking time series

In [216]:
df_results_cm_p["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [217]:
df_results_cm_p_btss = (
    df_results_cm_p[
        df_results_cm_p["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [218]:
df_results_cm_p_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [219]:
df_results_cm_p.shape, df_results_cm_p_btss.shape

((40, 17), (20, 17))

<IPython.core.display.Javascript object>

###### SCPM computation

In [220]:
df_results_cm_p_btss = compute_scpm(df_results_cm_p_btss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [221]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_btss)
dominance_matrix_cm_p_btss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_btss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [222]:
dominance_matrix_cm_p_btss.shape, len(dominance_graph_cm_p_btss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [223]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    18
dominant_model         1
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [224]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/203_c_dominance_analysis_cm_p_btss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [225]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
15,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,Transformer,Transformer_1.0,1.0,1.113216,...,1.971579,0.963484,0.210617,0.094078,0.188811,0.011791,-5.269450,0,dominant_model,19
5,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Trees,Random Forest,Random Forest,NaN,1.164482,...,2.093257,0.956677,0.343249,0.222080,0.546564,0.022441,-5.004775,1,intermediate_model,18
16,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,Transformer,Transformer_7.0,7.0,1.352578,...,2.563343,0.945704,0.206991,0.136490,0.380527,0.011697,-4.056963,2,intermediate_model,16
19,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Linear Regression,Linear Regression,Linear Regression,NaN,1.397380,...,2.549150,0.942855,0.210220,0.139439,0.350685,0.016063,-3.972969,2,intermediate_model,16
4,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Trees,XGBoost,XGBoost,NaN,1.544001,...,3.046233,0.928582,0.254903,0.156771,0.358178,0.026031,-3.050336,4,intermediate_model,12


<IPython.core.display.Javascript object>

In [226]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [227]:
# Top 5
df_sorted_topo_cols.iloc[0:5].sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
15,Transformer_1.0,1.11,0.21,0.82,0.09,1.97,0.19,0.96,0.01,-5.27,0,dominant_model,19,c
5,Random Forest,1.16,0.34,0.86,0.22,2.09,0.55,0.96,0.02,-5.00,1,intermediate_model,18,c
16,Transformer_7.0,1.35,0.21,1.04,0.14,2.56,0.38,0.95,0.01,-4.06,2,intermediate_model,16,c
19,Linear Regression,1.40,0.21,1.05,0.14,2.55,0.35,0.94,0.02,-3.97,2,intermediate_model,16,c
4,XGBoost,1.54,0.25,1.22,0.16,3.05,0.36,0.93,0.03,-3.05,4,intermediate_model,12,c


<IPython.core.display.Javascript object>

In [228]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [229]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
5,Random Forest,1.16,0.34,0.86,0.22,2.09,0.55,0.96,0.02,-5.00,1,intermediate_model,18,c
16,Transformer_7.0,1.35,0.21,1.04,0.14,2.56,0.38,0.95,0.01,-4.06,2,intermediate_model,16,c
19,Linear Regression,1.40,0.21,1.05,0.14,2.55,0.35,0.94,0.02,-3.97,2,intermediate_model,16,c


<IPython.core.display.Javascript object>

In [230]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(18, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [231]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominated_Count", "SCPM"], ascending=[True, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
7,BidirectionalLSTM_7.0,3.89,2.27,3.19,2.23,7.8,5.32,0.44,0.59,10.29,19,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [232]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [233]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [234]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [235]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [236]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [237]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression intermediate_model      100.00
Neural Networks   intermediate_model       87.50
                  dominant_model            6.25
                  non_dominant_model        6.25
Trees             intermediate_model      100.00

<IPython.core.display.Javascript object>

In [238]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                 2.0            10.25   3.00
                std                  NaN             5.41   1.73
                min                  2.0             0.00   1.00
                25%                  2.0             7.00   2.50
                50%                  2.0            11.50   4.00
                75%                  2.0            15.00   4.00
                max                  2.0            19.00   4.00
Dominates_Count count                1.0            16.00   3.00
                mean                16.0             7.31  14.00
                std                  NaN             5.80   3.46
                min                 16.0             0.00  12.00
                25%                 16.0             1.00  12.00
                50%                 16.0             7.00  12.00
                75%                 16.0            11.25  15.00
                max                 16.0            19.00  18.00

<IPython.core.display.Javascript object>

In [239]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,intermediate_model,1
Neural Networks,16,3,intermediate_model,14
Trees,3,1,intermediate_model,3


<IPython.core.display.Javascript object>

In [240]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  intermediate_model     1
Neural Networks    intermediate_model    14
                   dominant_model         1
                   non_dominant_model     1
Trees              intermediate_model     3
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [241]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [242]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,16.0000,16.0,2,16,0.80
Neural Networks,16,7.3125,7.0,164,117,5.85
Trees,3,14.0000,12.0,9,42,2.10


<IPython.core.display.Javascript object>

In [243]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  1.397380       NaN  1.397380  1.397380  1.397380   
Neural Networks        16.0  2.327602  0.723700  1.113216  1.865003  2.397739   
Trees                   3.0  1.454054  0.256703  1.164482  1.354242  1.544001   

                                      RMSE_std            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  1.397380  1.397380      1.0  0.210220  ...             2.0   
Neural Networks    2.742636  3.885565     16.0  0.872075  ...            15.0   
Trees              1.598840  1.653679      3.0  0.526040  ...             4.0   

                        Dominates_Count                                       \
                    max           count     mean       std   min   25%   50%   
Model                                                                          
Linear Regression   2.0             1.0  16.0000       NaN  16.0  16.0  16.0   
Neural Networks    19.0            16.0   7.3125  5.804811   0.0   1.0   7.0   
Trees               4.0             3.0  14.0000  3.464102  12.0  12.0  12.0   

                                
                     75%   max  
Model                           
Linear Regression  16.00  16.0  
Neural Networks    11.25  19.0  
Trees              15.00  18.0  

[3 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [244]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [245]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [246]:
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [247]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
15,Transformer_1.0,1.11,0.21,0.82,0.09,1.97,0.19,0.96,0.01,-5.27,0,19,dominant_model,c,True
5,Random Forest,1.16,0.34,0.86,0.22,2.09,0.55,0.96,0.02,-5.00,1,18,intermediate_model,c,True
16,Transformer_7.0,1.35,0.21,1.04,0.14,2.56,0.38,0.95,0.01,-4.06,2,16,intermediate_model,c,True
19,Linear Regression,1.40,0.21,1.05,0.14,2.55,0.35,0.94,0.02,-3.97,2,16,intermediate_model,c,True
12,BidirectionalGRU_1.0,1.55,0.48,1.16,0.29,2.72,0.67,0.93,0.04,-3.34,4,12,intermediate_model,c,False


<IPython.core.display.Javascript object>

### Time Series Split

In [248]:
df_results_cm_p["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [249]:
df_results_cm_p_tss = (
    df_results_cm_p[df_results_cm_p["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [250]:
df_results_cm_p_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [251]:
df_results_cm_p_tss.shape

(20, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [252]:
df_results_cm_p_tss = compute_scpm(df_results_cm_p_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [253]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_tss)
dominance_matrix_cm_p_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [254]:
dominance_matrix_cm_p_tss.shape, len(dominance_graph_cm_p_tss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [255]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    15
non_dominant_model     3
dominant_model         2
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [256]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/203_c_dominance_analysis_cm_p_tss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [257]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
5,203,c,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Trees,Random Forest,Random Forest,NaN,1.930476,...,3.111834,0.869324,1.050019,0.645549,1.302312,0.129117,-5.402798,0,dominant_model,18
16,203,c,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,Transformer,Transformer_7.0,7.0,1.968269,...,3.547691,0.879904,0.476715,0.338001,0.726416,0.059485,-4.530218,0,dominant_model,18
11,203,c,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,LSTM,LSTM_14.0,14.0,2.385303,...,3.581665,0.832393,0.349957,0.290923,0.634890,0.039929,-3.928519,2,intermediate_model,16
15,203,c,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,Transformer,Transformer_1.0,1.0,2.168878,...,3.832677,0.854362,0.666195,0.503445,1.119284,0.077029,-3.600525,2,intermediate_model,15
18,203,c,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Trees,Decision Tree,Decision Tree,NaN,2.416298,...,3.705620,0.791749,1.308648,0.661583,1.349594,0.179707,-3.486942,3,intermediate_model,13


<IPython.core.display.Javascript object>

In [258]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [259]:
# Top 5
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[0:5].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
5,Random Forest,1.93,1.05,1.37,0.65,3.11,1.30,0.87,0.13,-5.40,0,dominant_model,18,c
16,Transformer_7.0,1.97,0.48,1.49,0.34,3.55,0.73,0.88,0.06,-4.53,0,dominant_model,18,c
11,LSTM_14.0,2.39,0.35,1.55,0.29,3.58,0.63,0.83,0.04,-3.93,2,intermediate_model,16,c
15,Transformer_1.0,2.17,0.67,1.64,0.50,3.83,1.12,0.85,0.08,-3.60,2,intermediate_model,15,c
18,Decision Tree,2.42,1.31,1.63,0.66,3.71,1.35,0.79,0.18,-3.49,3,intermediate_model,13,c


<IPython.core.display.Javascript object>

In [260]:
# Top 4
df_sorted_topo_cols.iloc[0:5].sort_values(by="Dominates_Count", ascending=False)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
5,Random Forest,1.93,1.05,1.37,0.65,3.11,1.30,0.87,0.13,-5.40,0,dominant_model,18,c
16,Transformer_7.0,1.97,0.48,1.49,0.34,3.55,0.73,0.88,0.06,-4.53,0,dominant_model,18,c
11,LSTM_14.0,2.39,0.35,1.55,0.29,3.58,0.63,0.83,0.04,-3.93,2,intermediate_model,16,c
15,Transformer_1.0,2.17,0.67,1.64,0.50,3.83,1.12,0.85,0.08,-3.60,2,intermediate_model,15,c
18,Decision Tree,2.42,1.31,1.63,0.66,3.71,1.35,0.79,0.18,-3.49,3,intermediate_model,13,c


<IPython.core.display.Javascript object>

In [261]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(2, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [262]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
11,LSTM_14.0,2.39,0.35,1.55,0.29,3.58,0.63,0.83,0.04,-3.93,2,intermediate_model,16,c
15,Transformer_1.0,2.17,0.67,1.64,0.50,3.83,1.12,0.85,0.08,-3.60,2,intermediate_model,15,c
18,Decision Tree,2.42,1.31,1.63,0.66,3.71,1.35,0.79,0.18,-3.49,3,intermediate_model,13,c


<IPython.core.display.Javascript object>

In [263]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(15, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [264]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:5]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
8,BidirectionalLSTM_14.0,4.57,3.02,2.35,0.48,5.46,1.12,0.14,1.13,3.16,15,non_dominant_model,0,c
9,LSTM_1.0,4.32,1.83,2.90,1.23,6.45,2.57,0.36,0.50,5.34,16,non_dominant_model,0,c
3,GRU_14.0,6.78,8.94,2.45,1.44,5.42,3.04,-2.62,6.92,8.76,16,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [265]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(3, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [266]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [267]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [268]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [269]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [270]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression intermediate_model      100.00
Neural Networks   intermediate_model       75.00
                  non_dominant_model       18.75
                  dominant_model            6.25
Trees             intermediate_model       66.67
                  dominant_model           33.33

<IPython.core.display.Javascript object>

In [271]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                 7.0             8.88   2.33
                std                  NaN             5.19   2.08
                min                  7.0             0.00   0.00
                25%                  7.0             6.25   1.50
                50%                  7.0             8.50   3.00
                75%                  7.0            12.75   3.50
                max                  7.0            16.00   4.00
Dominates_Count count                1.0            16.00   3.00
                mean                 8.0             6.50  14.67
                std                  NaN             6.06   2.89
                min                  8.0             0.00  13.00
                25%                  8.0             1.00  13.00
                50%                  8.0             4.50  13.00
                75%                  8.0             9.25  15.50
                max                  8.0            18.00  18.00

<IPython.core.display.Javascript object>

In [272]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [273]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,intermediate_model,1
Neural Networks,16,3,intermediate_model,12
Trees,3,2,intermediate_model,2


<IPython.core.display.Javascript object>

In [274]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  intermediate_model     1
Neural Networks    intermediate_model    12
                   non_dominant_model     3
                   dominant_model         1
Trees              intermediate_model     2
                   dominant_model         1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [275]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [276]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,8.00,8.0,7,8,0.4
Neural Networks,16,6.50,4.5,142,104,5.2
Trees,3,14.67,13.0,7,44,2.2


<IPython.core.display.Javascript object>

In [277]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  2.787250       NaN  2.787250  2.787250  2.787250   
Neural Networks        16.0  3.567874  1.288881  1.968269  2.772547  3.282451   
Trees                   3.0  2.264681  0.289844  1.930476  2.173387  2.416298   

                                      RMSE_std            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  2.787250  2.787250      1.0  0.515494  ...            7.00   
Neural Networks    4.001532  6.775743     16.0  1.778408  ...           12.75   
Trees              2.431783  2.447268      3.0  1.154860  ...            3.50   

                        Dominates_Count                                   \
                    max           count       mean       std   min   25%   
Model                                                                      
Linear Regression   7.0             1.0   8.000000       NaN   8.0   8.0   
Neural Networks    16.0            16.0   6.500000  6.055301   0.0   1.0   
Trees               4.0             3.0  14.666667  2.886751  13.0  13.0   

                                      
                    50%    75%   max  
Model                                 
Linear Regression   8.0   8.00   8.0  
Neural Networks     4.5   9.25  18.0  
Trees              13.0  15.50  18.0  

[3 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [278]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [279]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [280]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [281]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
5,Random Forest,1.93,1.05,1.37,0.65,3.11,1.30,0.87,0.13,-5.40,0,18,dominant_model,c,True
16,Transformer_7.0,1.97,0.48,1.49,0.34,3.55,0.73,0.88,0.06,-4.53,0,18,dominant_model,c,True
11,LSTM_14.0,2.39,0.35,1.55,0.29,3.58,0.63,0.83,0.04,-3.93,2,16,intermediate_model,c,True
15,Transformer_1.0,2.17,0.67,1.64,0.50,3.83,1.12,0.85,0.08,-3.60,2,15,intermediate_model,c,True
18,Decision Tree,2.42,1.31,1.63,0.66,3.71,1.35,0.79,0.18,-3.49,3,13,intermediate_model,c,True


<IPython.core.display.Javascript object>

### Variable Grouping: CM-P-CS

In [282]:
df_results["Features"].unique()

array(['Chemical + Mineralogical + Physical', 'Chemical + Mineralogical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [283]:
df_results_cm_p_cs = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [284]:
df_results_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [285]:
df_results_cm_p.shape

(40, 17)

<IPython.core.display.Javascript object>

### Blocking time series

In [286]:
df_results_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [287]:
df_results_cm_p_cs_btss = (
    df_results_cm_p_cs[
        df_results_cm_p_cs["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [288]:
df_results_cm_p_cs_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [289]:
df_results_cm_p_cs.shape, df_results_cm_p_cs_btss.shape

((37, 17), (20, 17))

<IPython.core.display.Javascript object>

###### SCPM computation

In [290]:
df_results_cm_p_cs_btss = compute_scpm(df_results_cm_p_cs_btss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [291]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_btss)
dominance_matrix_cm_p_cs_btss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_btss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [292]:
dominance_matrix_cm_p_cs_btss.shape, len(dominance_graph_cm_p_cs_btss)

((20, 20), 20)

<IPython.core.display.Javascript object>

In [293]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    18
dominant_model         1
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [294]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/203_c_dominance_analysis_cm_p_cs_btss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [295]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Trees,Random Forest,Random Forest,NaN,0.876747,...,1.544938,0.977017,0.192428,0.154084,0.326348,0.008537,-6.358553,0,dominant_model,19
11,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,Transformer,Transformer_1.0,1.0,0.999045,...,1.861475,0.971209,0.060993,0.057925,0.154027,0.002508,-5.719730,1,intermediate_model,18
5,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Trees,Decision Tree,Decision Tree,NaN,1.150395,...,2.069280,0.952045,0.512101,0.372709,0.955340,0.042966,-5.079948,2,intermediate_model,16
9,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Linear Regression,Linear Regression,Linear Regression,NaN,1.112537,...,2.058687,0.964267,0.080123,0.073386,0.185810,0.004084,-5.229151,2,intermediate_model,16
10,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Trees,XGBoost,XGBoost,NaN,1.489677,...,2.980154,0.935297,0.127922,0.114261,0.189484,0.012392,-3.298618,4,intermediate_model,15


<IPython.core.display.Javascript object>

In [296]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [297]:
# Top 5
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[0:5].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,Random Forest,0.88,0.19,0.66,0.15,1.54,0.33,0.98,0.01,-6.36,0,dominant_model,19,c
11,Transformer_1.0,1.00,0.06,0.78,0.06,1.86,0.15,0.97,0.00,-5.72,1,intermediate_model,18,c
9,Linear Regression,1.11,0.08,0.87,0.07,2.06,0.19,0.96,0.00,-5.23,2,intermediate_model,16,c
5,Decision Tree,1.15,0.51,0.86,0.37,2.07,0.96,0.95,0.04,-5.08,2,intermediate_model,16,c
10,XGBoost,1.49,0.13,1.19,0.11,2.98,0.19,0.94,0.01,-3.30,4,intermediate_model,15,c


<IPython.core.display.Javascript object>

In [298]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [299]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
11,Transformer_1.0,1.00,0.06,0.78,0.06,1.86,0.15,0.97,0.00,-5.72,1,intermediate_model,18,c
9,Linear Regression,1.11,0.08,0.87,0.07,2.06,0.19,0.96,0.00,-5.23,2,intermediate_model,16,c
5,Decision Tree,1.15,0.51,0.86,0.37,2.07,0.96,0.95,0.04,-5.08,2,intermediate_model,16,c


<IPython.core.display.Javascript object>

In [300]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(18, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [301]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominated_Count", "SCPM"], ascending=[True, True]).iloc[:6]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
18,BidirectionalLSTM_7.0,3.73,2.0,2.94,1.98,7.11,4.53,0.5,0.52,9.28,19,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [302]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [303]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [304]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      16
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [305]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [306]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [307]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression intermediate_model      100.00
Neural Networks   intermediate_model       93.75
                  non_dominant_model        6.25
Trees             intermediate_model       66.67
                  dominant_model           33.33

<IPython.core.display.Javascript object>

In [308]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            16.00   3.00
                mean                 2.0            10.56   2.00
                std                  NaN             5.12   2.00
                min                  2.0             1.00   0.00
                25%                  2.0             7.00   1.00
                50%                  2.0             9.50   2.00
                75%                  2.0            15.00   3.00
                max                  2.0            19.00   4.00
Dominates_Count count                1.0            16.00   3.00
                mean                16.0             6.94  16.67
                std                  NaN             5.03   2.08
                min                 16.0             0.00  15.00
                25%                 16.0             2.75  15.50
                50%                 16.0             6.50  16.00
                75%                 16.0            10.25  17.50
                max                 16.0            18.00  19.00

<IPython.core.display.Javascript object>

In [309]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,intermediate_model,1
Neural Networks,16,2,intermediate_model,15
Trees,3,2,intermediate_model,2


<IPython.core.display.Javascript object>

In [310]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  intermediate_model     1
Neural Networks    intermediate_model    15
                   non_dominant_model     1
Trees              intermediate_model     2
                   dominant_model         1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [311]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [312]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [313]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,16.000000,16.0,2,16,0.80
Neural Networks,16,6.937500,6.5,169,111,5.55
Trees,3,16.666667,16.0,6,50,2.50


<IPython.core.display.Javascript object>

In [314]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  1.112537       NaN  1.112537  1.112537  1.112537   
Neural Networks        16.0  2.386522  0.651324  0.999045  2.074538  2.380414   
Trees                   3.0  1.172273  0.307050  0.876747  1.013571  1.150395   

                                      RMSE_std            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  1.112537  1.112537      1.0  0.080123  ...             2.0   
Neural Networks    2.757541  3.733490     16.0  0.751028  ...            15.0   
Trees              1.320036  1.489677      3.0  0.277483  ...             3.0   

                        Dominates_Count                                    \
                    max           count       mean       std   min    25%   
Model                                                                       
Linear Regression   2.0             1.0  16.000000       NaN  16.0  16.00   
Neural Networks    19.0            16.0   6.937500  5.026181   0.0   2.75   
Trees               4.0             3.0  16.666667  2.081666  15.0  15.50   

                                      
                    50%    75%   max  
Model                                 
Linear Regression  16.0  16.00  16.0  
Neural Networks     6.5  10.25  18.0  
Trees              16.0  17.50  19.0  

[3 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [315]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [316]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [317]:
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [318]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(5).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
0,Random Forest,0.87675,0.19243,0.65788,0.15408,1.54494,0.32635,0.97702,0.00854,-6.35855,0,19,dominant_model,c,True
11,Transformer_1.0,0.99905,0.06099,0.78292,0.05792,1.86148,0.15403,0.97121,0.00251,-5.71973,1,18,intermediate_model,c,True
9,Linear Regression,1.11254,0.08012,0.86923,0.07339,2.05869,0.18581,0.96427,0.00408,-5.22915,2,16,intermediate_model,c,False
5,Decision Tree,1.15040,0.51210,0.85931,0.37271,2.06928,0.95534,0.95205,0.04297,-5.07995,2,16,intermediate_model,c,False
10,XGBoost,1.48968,0.12792,1.19444,0.11426,2.98015,0.18948,0.93530,0.01239,-3.29862,4,15,intermediate_model,c,True


<IPython.core.display.Javascript object>

### Time Series Split

In [319]:
df_results_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [320]:
df_results_cm_p_cs_tss = (
    df_results_cm_p_cs[df_results_cm_p_cs["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [321]:
df_results_cm_p_cs_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [322]:
df_results_cm_p_cs_tss.shape

(17, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [323]:
df_results_cm_p_cs_tss = compute_scpm(df_results_cm_p_cs_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [324]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_tss)
dominance_matrix_cm_p_cs_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [325]:
dominance_matrix_cm_p_cs_tss.shape, len(dominance_graph_cm_p_cs_tss)

((17, 17), 17)

<IPython.core.display.Javascript object>

In [326]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    14
dominant_model         2
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [327]:
df_sorted_topo.to_csv(
    "../../../../reports/results/local_models/ecics/203_c_dominance_analysis_cm_p_cs_tss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [328]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,203,c,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Trees,Random Forest,Random Forest,NaN,1.469900,...,2.543769,0.929517,0.637965,0.357381,0.739222,0.058918,-3.879706,0,dominant_model,14
9,203,c,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Linear Regression,Linear Regression,Linear Regression,NaN,1.492822,...,2.558572,0.935135,0.285013,0.266661,0.598803,0.020116,-3.833748,0,dominant_model,15
11,203,c,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,Transformer,Transformer_1.0,1.0,1.517652,...,2.677090,0.929673,0.377983,0.272467,0.603415,0.032692,-3.564974,1,intermediate_model,14
5,203,c,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Trees,Decision Tree,Decision Tree,NaN,1.729078,...,2.933841,0.890315,1.055642,0.630465,1.306712,0.130899,-2.736717,3,intermediate_model,11
12,203,c,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,Transformer,Transformer_7.0,7.0,1.737521,...,3.181172,0.911078,0.147880,0.158667,0.372748,0.018718,-2.354102,3,intermediate_model,12


<IPython.core.display.Javascript object>

In [329]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [330]:
# Top 5
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[0:5].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
9,Linear Regression,1.49,0.29,1.11,0.27,2.56,0.60,0.94,0.02,-3.83,0,dominant_model,15,c
0,Random Forest,1.47,0.64,1.10,0.36,2.54,0.74,0.93,0.06,-3.88,0,dominant_model,14,c
11,Transformer_1.0,1.52,0.38,1.15,0.27,2.68,0.60,0.93,0.03,-3.56,1,intermediate_model,14,c
12,Transformer_7.0,1.74,0.15,1.34,0.16,3.18,0.37,0.91,0.02,-2.35,3,intermediate_model,12,c
5,Decision Tree,1.73,1.06,1.30,0.63,2.93,1.31,0.89,0.13,-2.74,3,intermediate_model,11,c


<IPython.core.display.Javascript object>

In [331]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(2, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [332]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
11,Transformer_1.0,1.52,0.38,1.15,0.27,2.68,0.60,0.93,0.03,-3.56,1,intermediate_model,14,c
12,Transformer_7.0,1.74,0.15,1.34,0.16,3.18,0.37,0.91,0.02,-2.35,3,intermediate_model,12,c
5,Decision Tree,1.73,1.06,1.30,0.63,2.93,1.31,0.89,0.13,-2.74,3,intermediate_model,11,c
13,Transformer_14.0,1.84,0.32,1.42,0.25,3.43,0.59,0.89,0.05,-1.80,4,intermediate_model,10,c


<IPython.core.display.Javascript object>

In [333]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(14, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [334]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominated_Count", "SCPM"], ascending=[True, False]).iloc[:5]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
8,BidirectionalGRU_14.0,7.75,10.25,2.58,1.74,5.79,3.65,-3.75,9.03,12.19,16,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [335]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [336]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [337]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks      13
Trees                 3
Linear Regression     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [338]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [339]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [340]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model             Classification                
Linear Regression dominant_model          100.00
Neural Networks   intermediate_model       92.31
                  non_dominant_model        7.69
Trees             intermediate_model       66.67
                  dominant_model           33.33

<IPython.core.display.Javascript object>

In [341]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Linear Regression  Neural Networks  Trees
Dominated_Count count                1.0            13.00   3.00
                mean                 0.0             7.92   3.00
                std                  NaN             4.41   3.00
                min                  0.0             1.00   0.00
                25%                  0.0             5.00   1.50
                50%                  0.0             7.00   3.00
                75%                  0.0            10.00   4.50
                max                  0.0            16.00   6.00
Dominates_Count count                1.0            13.00   3.00
                mean                15.0             5.15  10.00
                std                  NaN             4.45   4.58
                min                 15.0             0.00   5.00
                25%                 15.0             2.00   8.00
                50%                 15.0             3.00  11.00
                75%                 15.0             7.00  12.50
                max                 15.0            14.00  14.00

<IPython.core.display.Javascript object>

In [342]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [343]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Linear Regression,1,1,dominant_model,1
Neural Networks,13,2,intermediate_model,12
Trees,3,2,intermediate_model,2


<IPython.core.display.Javascript object>

In [344]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model              Classification    
Linear Regression  dominant_model         1
Neural Networks    intermediate_model    12
                   non_dominant_model     1
Trees              intermediate_model     2
                   dominant_model         1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [345]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [346]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Linear Regression,1,15.00,15.0,0,15,0.88
Neural Networks,13,5.15,3.0,103,67,3.94
Trees,3,10.00,11.0,9,30,1.76


<IPython.core.display.Javascript object>

In [347]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                      count      mean       std       min       25%       50%   
Model                                                                           
Linear Regression       1.0  1.492822       NaN  1.492822  1.492822  1.492822   
Neural Networks        13.0  3.034903  1.594949  1.517652  2.134392  2.717913   
Trees                   3.0  1.764142  0.313249  1.469900  1.599489  1.729078   

                                      RMSE_std            ... Dominated_Count  \
                        75%       max    count      mean  ...             75%   
Model                                                     ...                   
Linear Regression  1.492822  1.492822      1.0  0.285013  ...             0.0   
Neural Networks    3.348556  7.745173     13.0  1.857645  ...            10.0   
Trees              1.911263  2.093448      3.0  0.917052  ...             4.5   

                        Dominates_Count                                   \
                    max           count       mean       std   min   25%   
Model                                                                      
Linear Regression   0.0             1.0  15.000000       NaN  15.0  15.0   
Neural Networks    16.0            13.0   5.153846  4.450583   0.0   2.0   
Trees               6.0             3.0  10.000000  4.582576   5.0   8.0   

                                     
                    50%   75%   max  
Model                                
Linear Regression  15.0  15.0  15.0  
Neural Networks     3.0   7.0  14.0  
Trees              11.0  12.5  14.0  

[3 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [348]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [349]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [350]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [351]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
0,Random Forest,1.47,0.64,1.10,0.36,2.54,0.74,0.93,0.06,-3.88,0,14,dominant_model,c,True
9,Linear Regression,1.49,0.29,1.11,0.27,2.56,0.60,0.94,0.02,-3.83,0,15,dominant_model,c,True
11,Transformer_1.0,1.52,0.38,1.15,0.27,2.68,0.60,0.93,0.03,-3.56,1,14,intermediate_model,c,True
5,Decision Tree,1.73,1.06,1.30,0.63,2.93,1.31,0.89,0.13,-2.74,3,11,intermediate_model,c,True
12,Transformer_7.0,1.74,0.15,1.34,0.16,3.18,0.37,0.91,0.02,-2.35,3,12,intermediate_model,c,True


<IPython.core.display.Javascript object>